In [0]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/node-master'       # path to your project on Drive

GIT_USERNAME = "sgsyang" # replace with yours
#GIT_TOKEN = "XXX"           # definitely replace with yours
#GIT_REPOSITORY = "yyy"      # ...nah


drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
GIT_PATH = "https://github.com/sgsyang/node.git"

!mkdir ./temp
!git clone "{GIT_PATH}"
!mv ./temp/* "{PROJECT_PATH}"
!rm -rf ./temp
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/My Drive/node-master’: File exists
fatal: destination path 'node' already exists and is not an empty directory.
mv: cannot stat './temp/*': No such file or directory
sending incremental file list
lib/
notebooks/Classification/Adult_Catoost.ipynb
        463,122 100%  410.42MB/s    0:00:00 (xfr#1, to-chk=17/36)
notebooks/Classification/Adult_Xgboost.ipynb
         61,109 100%  192.51kB/s    0:00:00 (xfr#2, to-chk=16/36)
notebooks/Regression/Year_CatBoost.ipynb
        397,040 100%  603.95kB/s    0:00:00 (xfr#3, to-chk=1/36)


In [0]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0,1
import os, sys
import time
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
!pip install catboost
!pip install category_encoders
!pip install tensorboardX
# !pip install qhoptim



import lib
import torch, torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

experiment_name = 'Catboost'
experiment_name = '{}_{}.{:0>2d}.{:0>2d}_{:0>2d}:{:0>2d}'.format(experiment_name, *time.gmtime()[:5])
print("experiment:", experiment_name)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CUDA_VISIBLE_DEVICES=0,1
experiment: Catboost_2020.05.27_16:03


In [0]:
torch.cuda.get_device_name(device)

'Tesla P100-PCIE-16GB'

## Default Parameters

In [0]:
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

# 13, 1, 602, 295, 970, 2000, 3000, 10000
random_state = [1337, 133,13, 1, 602, 295, 970, 2000, 3000, 10000]
def Test(random_state):
  data = lib.Dataset("YEAR", random_state=random_state, quantile_transform=True, quantile_noise=1e-3)
  in_features = data.X_train.shape[1]

  mu, std = data.y_train.mean(), data.y_train.std()
  normalize = lambda x: ((x - mu) / std).astype(np.float32)
  data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])

  model = CatBoostRegressor(objective = 'RMSE',task_type="GPU",
                           devices='0:1')
  model.fit(data.X_train,data.y_train)


  test_preds = model.predict(data.X_test)
  test_mse = mean_squared_error(data.y_test, test_preds)
  # print('Catboost for Year Prediction')

  score = test_mse * std ** 2
  print('-----')
  print('The random_state:',random_state)
  print('The Score',score)
  print(f'Test{random_state} is Over!!!')
  return score, random_state


score_results = [] 
key_results = []


for i in random_state:
  score, random_state = Test(i)
  score_results.append(score)
  key_results.append(random_state)




流式输出内容被截断，只能显示最后 5000 行内容。
25:	learn: 0.8586461	total: 554ms	remaining: 20.8s
26:	learn: 0.8572955	total: 572ms	remaining: 20.6s
27:	learn: 0.8561308	total: 593ms	remaining: 20.6s
28:	learn: 0.8551933	total: 611ms	remaining: 20.5s
29:	learn: 0.8541905	total: 630ms	remaining: 20.4s
30:	learn: 0.8532664	total: 647ms	remaining: 20.2s
31:	learn: 0.8523019	total: 663ms	remaining: 20.1s
32:	learn: 0.8514295	total: 679ms	remaining: 19.9s
33:	learn: 0.8506293	total: 695ms	remaining: 19.8s
34:	learn: 0.8499465	total: 716ms	remaining: 19.7s
35:	learn: 0.8491869	total: 732ms	remaining: 19.6s
36:	learn: 0.8486323	total: 749ms	remaining: 19.5s
37:	learn: 0.8480115	total: 767ms	remaining: 19.4s
38:	learn: 0.8472312	total: 783ms	remaining: 19.3s
39:	learn: 0.8465619	total: 800ms	remaining: 19.2s
40:	learn: 0.8459331	total: 816ms	remaining: 19.1s
41:	learn: 0.8453215	total: 831ms	remaining: 19s
42:	learn: 0.8448616	total: 848ms	remaining: 18.9s
43:	learn: 0.8443140	total: 863ms	remaining: 18.7s
44:	le

In [0]:
import pprint
Result= dict(zip(key_results,score_results))
pprint.pprint(Result, width=1)

{1: 80.3827416996541,
 13: 80.31187063107521,
 133: 80.38105671723287,
 295: 80.34674133330854,
 602: 80.40662791044201,
 970: 80.16362715502423,
 1337: 80.35091267569976,
 2000: 80.53033444071298,
 3000: 80.44718868487982,
 10000: 80.42716483579963}


In [0]:
# random_state = 1337, 133, 13, 1, 602, 295, 970, 2000, 3000, 10000

# random_state = 13

# data = lib.Dataset("YEAR", random_state=random_state, quantile_transform=True, quantile_noise=1e-3)
# in_features = data.X_train.shape[1]

# mu, std = data.y_train.mean(), data.y_train.std()
# normalize = lambda x: ((x - mu) / std).astype(np.float32)
# data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])

# print("mean = %.5f, std = %.5f" % (mu, std))



mean = 1998.39193, std = 10.92832


In [0]:

# from sklearn.metrics import mean_squared_error
# from catboost import CatBoostRegressor
# model = CatBoostRegressor(objective = 'RMSE',task_type="GPU",
#                            devices='0:1')
# model.fit(data.X_train,data.y_train)

# valid_preds = model.predict(data.X_valid)

# valid_mse = mean_squared_error(data.y_valid, valid_preds)
# print("MSE of Validation: %f" % (valid_mse))


Learning rate set to 0.171501
0:	learn: 0.9770754	total: 24.7ms	remaining: 24.7s
1:	learn: 0.9594189	total: 48ms	remaining: 23.9s
2:	learn: 0.9448029	total: 70.1ms	remaining: 23.3s
3:	learn: 0.9333009	total: 95.8ms	remaining: 23.9s
4:	learn: 0.9239425	total: 117ms	remaining: 23.4s
5:	learn: 0.9164391	total: 140ms	remaining: 23.2s
6:	learn: 0.9089866	total: 162ms	remaining: 23s
7:	learn: 0.9032817	total: 183ms	remaining: 22.7s
8:	learn: 0.8985944	total: 207ms	remaining: 22.8s
9:	learn: 0.8939182	total: 226ms	remaining: 22.4s
10:	learn: 0.8900849	total: 247ms	remaining: 22.2s
11:	learn: 0.8868202	total: 266ms	remaining: 21.9s
12:	learn: 0.8836963	total: 285ms	remaining: 21.6s
13:	learn: 0.8805586	total: 310ms	remaining: 21.8s
14:	learn: 0.8777576	total: 331ms	remaining: 21.7s
15:	learn: 0.8750670	total: 349ms	remaining: 21.5s
16:	learn: 0.8730257	total: 377ms	remaining: 21.8s
17:	learn: 0.8709083	total: 399ms	remaining: 21.8s
18:	learn: 0.8692909	total: 419ms	remaining: 21.6s
19:	learn: 

In [0]:

# test_preds = model.predict(data.X_test)


# test_mse = mean_squared_error(data.y_test, test_preds)
# print('Catboost for Year Prediction')
# print("MSE of Validation: %f" % (valid_mse))
# print("MSE of Test: %f" % (test_mse))

# print('The random_state:',random_state)
# print('The Score',test_mse * std ** 2)

Catboost for Year Prediction
MSE of Validation: 0.651875
MSE of Test: 0.672470
The random_state: 13
The Score 80.31187063107521


##Tuned Parameters

In [0]:
!pip install hyperopt
!pip install catboost

import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor


# paper_space={'learning_rate':hp.loguniform('learning_rate',np.exp(-5) ,np.exp(-3)),
#        'random_strength':hp.quniform("random_strength", 1,20,1),
#        'one_hot_max_size':hp.quniform("one_hot_max_size", 0,25,1),
#        'l2_leaf_reg': hp.loguniform('l2_leaf_reg',1 ,10),
#        'bagging_temperature':hp.uniform('bagging_temperature', 0, 1),
#        'leaf_estimation_iterations':hp.quniform("leaf_estimation_iterations", 1,10,1)
#     }

space = {
    'learning_rate':     hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
    'random_strength':hp.quniform("random_strength", 1,20,1),
    'one_hot_max_size':hp.quniform("one_hot_max_size", 0,25,1),
    'l2_leaf_reg': hp.loguniform('l2_leaf_reg',1 ,10),
    'bagging_temperature':hp.uniform('bagging_temperature', 0, 1),
    'leaf_estimation_iterations':hp.quniform("leaf_estimation_iterations", 1,10,1) 
}


In [0]:
##Automate
def my_tuning(space):
    # reg=CatBoostRegressor(learning_rate=our_space['learning_rate'],
    #                       max_depth = our_space['max_depth'],
    #                       colsample_bylevel = our_space['colsample_bylevel'],
    #                       objective = 'RMSE',task_type="GPU",
    #                       devices='0:1' 
    #                       )
    reg=CatBoostRegressor( learning_rate=space['learning_rate'],
                          random_strength = int(space['random_strength']),
                          one_hot_max_size = int(space['one_hot_max_size']),
                          l2_leaf_reg = space['l2_leaf_reg'],
                          bagging_temperature = space['bagging_temperature'],
                          leaf_estimation_iterations = int(space['leaf_estimation_iterations']),
                          objective = 'RMSE',task_type="GPU",
                          devices='0:1')
        
    reg.fit(data.X_train, data.y_train,verbose = 999)

    pred = reg.predict(data.X_test)
    mse= mean_squared_error(data.y_test, pred)
    print ("test loss:", mse)
    #change the metric if you like
    return {'loss':mse, 'status': STATUS_OK }



score_results = [] 
key_results = []

random_state = [1337, 133,13, 1, 602, 295, 970, 2000, 3000, 10000]
for i in random_state:
  data = lib.Dataset("YEAR", random_state=i, quantile_transform=True, quantile_noise=1e-3)
  in_features = data.X_train.shape[1]
  mu, std = data.y_train.mean(), data.y_train.std()
  normalize = lambda x: ((x - mu) / std).astype(np.float32)
  data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])
  print(f'The dataset with random state {i} is used')

  trials = Trials()
  best = fmin(fn=my_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
  # print('Best parameters',best)
  tuned_loss = trials.best_trial['result']['loss']
  print('Best Loss',tuned_loss)
  final = tuned_loss
  print('FINAL Error:',final)
  score_results.append(final)
  key_results.append(i)



The dataset with random state 1337 is used
0:	learn: 0.9836556	total: 9.97ms	remaining: 9.96s

999:	learn: 0.7920222	total: 8.37s	remaining: 0us

test loss:
0.671199831183972
0:	learn: 0.9872301	total: 12.1ms	remaining: 12.1s

999:	learn: 0.7776897	total: 10s	remaining: 0us

test loss:
0.6717915223497319
0:	learn: 0.9756684	total: 11.3ms	remaining: 11.3s

999:	learn: 0.7526793	total: 11s	remaining: 0us

test loss:
0.6723467095019271
0:	learn: 0.9660303	total: 9.26ms	remaining: 9.25s

999:	learn: 0.7368939	total: 8.45s	remaining: 0us

test loss:
0.6739230864968188
0:	learn: 0.9939913	total: 10.1ms	remaining: 10.1s

999:	learn: 0.8017518	total: 8.84s	remaining: 0us

test loss:
0.6770628182370805
0:	learn: 0.9934654	total: 10.3ms	remaining: 10.3s

999:	learn: 0.7950655	total: 9.37s	remaining: 0us

test loss:
0.6772024122195504
0:	learn: 0.9703213	total: 10.7ms	remaining: 10.7s

999:	learn: 0.7353816	total: 10s	remaining: 0us

test loss:
0.676677999787525
0:	learn: 0.9706945	total: 11.9ms	

In [0]:
import pprint
Result= dict(zip(key_results,score_results))

print('----')
pprint.pprint(Result, width=1)

mean = sum(score_results) / len(score_results)
print('Mean:',mean)
print('Max:',max(score_results))
print('Min:',min(score_results))
print('Distance to max',max(score_results)-mean)
print('Distance to min:',min(score_results)-mean)

----
{1: 0.6674163518138689,
 13: 0.6676847219534295,
 133: 0.6675242421734247,
 295: 0.6674998577761496,
 602: 0.6670427622357006,
 970: 0.6685488101113912,
 1337: 0.6672888947724297,
 2000: 0.6668896491549267,
 3000: 0.6676342160123043,
 10000: 0.667616576937497}
Mean: 0.6675146082941121
Max: 0.6685488101113912
Min: 0.6668896491549267
Distance to max 0.00103420181727909
Distance to min: -0.000624959139185477


TypeError: ignored

In [0]:
# std = 10.9283205039482
final_Score = []
x = 0
for i in score_results:
  x = i*std**2
  final_Score.append(x)



print('----')
Score_Result= dict(zip(key_results,final_Score))
pprint.pprint(Score_Result, width=1)

mean = sum(final_Score) / len(final_Score)
print('Mean:',mean)
print('Max:',max(final_Score))
print('Min:',min(final_Score))
print('Distance to max',max(final_Score)-mean)
print('Distance to min:',min(final_Score)-mean)


----
{1: 79.7083262308214,
 13: 79.74037719058073,
 133: 79.7212113810777,
 295: 79.71829919667037,
 602: 79.66370910405766,
 970: 79.84357367445443,
 1337: 79.69310426718229,
 2000: 79.64542308610295,
 3000: 79.73434535749654,
 10000: 79.73223875473602}
Mean: 79.72006082431801
Max: 79.84357367445443
Min: 79.64542308610295
Distance to max 0.12351285013642155
Distance to min: -0.07463773821505981


In [0]:
# data = lib.Dataset("YEAR", random_state=1337, quantile_transform=True, quantile_noise=1e-3)
# in_features = data.X_train.shape[1]
# mu, std = data.y_train.mean(), data.y_train.std()


# print('1337',std)
# data = lib.Dataset("YEAR", random_state=133, quantile_transform=True, quantile_noise=1e-3)
# in_features = data.X_train.shape[1]
# mu, std2 = data.y_train.mean(), data.y_train.std()

# print('133',std)

1337 10.9283205039482
133 10.9283205039482


In [0]:
## The following is used for single test


#Read the data
# ran_state=1337
# data = lib.Dataset("YEAR", random_state=ran_state, quantile_transform=True, quantile_noise=1e-3)
# in_features = data.X_train.shape[1]
# mu, std = data.y_train.mean(), data.y_train.std()
# normalize = lambda x: ((x - mu) / std).astype(np.float32)
# data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])
# print(f'The dataset with random state {ran_state} is used')

100%|██████████| 448576698/448576698 [01:10<00:00, 6333413.45it/s]


100%|██████████| 2507989/2507989 [00:00<00:00, 10232363.07it/s]


100%|██████████| 626904/626904 [00:00<00:00, 1117876.69it/s]


The dataset with random state 1337 is used


In [0]:



# Regression: 
# def hyperparameter_tuning(space):
  
    # reg=CatBoostRegressor( learning_rate=space['learning_rate'],
    #                       random_strength = int(space['random_strength']),
    #                       one_hot_max_size = int(space['one_hot_max_size']),
    #                       l2_leaf_reg = space['l2_leaf_reg'],
    #                       bagging_temperature = space['bagging_temperature'],
    #                       leaf_estimation_iterations = int(space['leaf_estimation_iterations']),
    #                       objective = 'RMSE',task_type="GPU",
    #                       devices='0:1')
        
    # reg.fit(data.X_train, data.y_train,verbose = 333)

    # pred = reg.predict(data.X_test)
    # mse= mean_squared_error(data.y_test, pred)
    # print ("test loss:", mse)
    # #change the metric if you like
    # return {'loss':mse, 'status': STATUS_OK }


# trials = Trials()
# best = fmin(fn=hyperparameter_tuning,
#             space=space,
#             algo=tpe.suggest,
#             max_evals=50,
#             trials=trials)

# print('Best parameters',best)
# tuned_loss = trials.best_trial['result']['loss']
# final = tuned_loss* std ** 2
# print('FINAL Score:',final)


##79.78756556500545 1337

0:	learn: 0.9706407	total: 12.8ms	remaining: 12.8s

333:	learn: 0.7878109	total: 3.43s	remaining: 6.85s

666:	learn: 0.7610088	total: 6.75s	remaining: 3.37s

999:	learn: 0.7392700	total: 10s	remaining: 0us

test loss:
0.6722485645961962
0:	learn: 0.9624964	total: 17.3ms	remaining: 17.3s

333:	learn: 0.7903534	total: 3.05s	remaining: 6.08s

666:	learn: 0.7665689	total: 6s	remaining: 2.99s

999:	learn: 0.7471611	total: 8.94s	remaining: 0us

test loss:
0.6755043035982486
0:	learn: 0.9872385	total: 12ms	remaining: 12s

333:	learn: 0.8091971	total: 3.24s	remaining: 6.47s

666:	learn: 0.7894509	total: 6.41s	remaining: 3.2s

999:	learn: 0.7751242	total: 9.58s	remaining: 0us

test loss:
0.6688184309415661
0:	learn: 0.9872341	total: 11.3ms	remaining: 11.3s

333:	learn: 0.8087086	total: 3.2s	remaining: 6.39s

666:	learn: 0.7884230	total: 6.35s	remaining: 3.17s

999:	learn: 0.7733090	total: 9.53s	remaining: 0us

test loss:
0.6701400429287366
0:	learn: 0.9703573	total: 12.6ms	remaining: 12.6s

333